# Advanced Machine Learning
# Assignment 5: Hyperparameter Optimization

### Davide Brinati
### Matricola: 771458

In [1]:
import numpy as np
from time import time

from pyGPGO.covfunc import squaredExponential
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.surrogates.RandomForest import RandomForest
from pyGPGO.GPGO import GPGO
from pyGPGO.acquisition import Acquisition

import sklearn
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
x, y = datasets.load_wine(return_X_y=True)

In [3]:
x

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [4]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

Standardizzo il dataset

In [5]:
scaler = StandardScaler()
x_sc = scaler.fit_transform(x)

In [6]:
def compute_accuracy_SVC(C,gamma, coef0):
        ds = datasets.load_wine()
        clf = svm.SVC(kernel = 'sigmoid', C=C,gamma=gamma, coef0=coef0)     #kernel sigmoid
        scores = cross_val_score(clf, x_sc, y, cv=10)
        return (scores.mean())

In [7]:
# defining a dictionary of params with range values
param = {'C': ('cont', [0.1,5]),'gamma': ('cont', [0.1,10]), 'coef0': ('cont', [0.1, 10])} 

In [8]:
initialPoints = 30            #30 function initialized
furtherEvaluations = 120      #150 in totale

### Surrogate model: Gaussian Process
### Acquisition function: ExpectedImprovement

In [9]:
# creating a GP surrogate model with a Squared Exponantial covariance function,
# aka kernel
sexp = squaredExponential()
sur_model = GaussianProcess(sexp)

# setting the acquisition function  --> ExpectedImprovement
acq = Acquisition(mode="ExpectedImprovement")

# creating an object Bayesian Optimization
bo = GPGO(sur_model,acq,compute_accuracy_SVC,param,n_jobs=-1)

# random seed
np.random.seed(123)

## START TIME
start = time()
# Initialize the design with some (i.e. 3) initial random evaluations
bo._firstRun(n_eval=initialPoints)

for iter in range(furtherEvaluations):
    bo._optimizeAcq(method='L-BFGS-B', n_start=1)
    bo.updateGP()
## END TIME
end = time() - start

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


In [10]:
print('Execution time: {}\nBayesan Optimization Results: {}'.format(end, bo.getResult()))

Execution time: 13.722853422164917
Bayesan Optimization Results: (OrderedDict([('C', 2.4511829154396807), ('gamma', 0.1), ('coef0', 3.113198459874254)]), 0.8674750601995184)


### Surrogate model: Gaussian Process
### Acquisition function: ProbabilityImprovement

In [11]:
# creating a GP surrogate model with a Squared Exponantial covariance function,
# aka kernel
sexp = squaredExponential()
sur_model = GaussianProcess(sexp)

# setting the acquisition function
acq = Acquisition(mode="ProbabilityImprovement")

# creating an object Bayesian Optimization
bo = GPGO(sur_model,acq,compute_accuracy_SVC,param,n_jobs=-1)

# random seed
np.random.seed(123)


## START TIME
start = time()
# Initialize the design with some (i.e. 3) initial random evaluations
bo._firstRun(n_eval=initialPoints)

for iter in range(furtherEvaluations):
    bo._optimizeAcq(method='L-BFGS-B', n_start=1)
    bo.updateGP()
## END TIME
end = time() - start

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


In [12]:
print('Execution time: {}\nBayesan Optimization Results: {}'.format(end, bo.getResult()))

Execution time: 10.66264533996582
Bayesan Optimization Results: (OrderedDict([('C', 3.5126990094295225), ('gamma', 2.932779416008757), ('coef0', 2.345829390285611)]), 0.8379815961472307)


### Surrogate model: Gaussian Process
### Acquisition function: UCB

In [13]:
# creating a GP surrogate model with a Squared Exponantial covariance function,
# aka kernel
sexp = squaredExponential()
sur_model = GaussianProcess(sexp)

# setting the acquisition function
acq = Acquisition(mode="UCB")

# creating an object Bayesian Optimization
bo = GPGO(sur_model,acq,compute_accuracy_SVC,param,n_jobs=-1)

# random seed
np.random.seed(123)


## START TIME
start = time()
# Initialize the design with some (i.e. 3) initial random evaluations
bo._firstRun(n_eval=initialPoints)

for iter in range(furtherEvaluations):
    bo._optimizeAcq(method='L-BFGS-B', n_start=1)
    bo.updateGP()
## END TIME
end = time() - start

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


In [14]:
print('Execution time: {}\nBayesan Optimization Results: {}'.format(end, bo.getResult()))

Execution time: 7.199481248855591
Bayesan Optimization Results: (OrderedDict([('C', 0.1), ('gamma', 8.518300653685488), ('coef0', 4.185212883157093)]), 0.8703280873753009)


### Surrogate model: Random Forest
### Acquisition function: ExpectedImprovement

In [15]:
# creating a GP surrogate model with a Squared Exponantial covariance function,
# aka kernel
sur_model = RandomForest()

# setting the acquisition function
acq = Acquisition(mode="ExpectedImprovement")

# creating an object Bayesian Optimization
bo = GPGO(sur_model,acq,compute_accuracy_SVC,param,n_jobs=-1)

# random seed
np.random.seed(123)


## START TIME
start = time()
# Initialize the design with some (i.e. 3) initial random evaluations
bo._firstRun(n_eval=initialPoints)

for iter in range(furtherEvaluations):
    bo._optimizeAcq(method='L-BFGS-B', n_start=1)
    bo.updateGP()
## END TIME
end = time() - start

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

In [16]:
print('Execution time: {}\nBayesan Optimization Results: {}'.format(end, bo.getResult()))

Execution time: 6.079193830490112
Bayesan Optimization Results: (OrderedDict([('C', 0.10058849729597702), ('gamma', 8.235954788764426), ('coef0', 3.626774143406337)]), 0.8700012899896802)


### Surrogate model: Random Forest
### Acquisition function: ProbabilityImprovement

In [17]:
# creating a GP surrogate model with a Squared Exponantial covariance function,
# aka kernel
sur_model = RandomForest()

# setting the acquisition function
acq = Acquisition(mode="ProbabilityImprovement")

# creating an object Bayesian Optimization
bo = GPGO(sur_model,acq,compute_accuracy_SVC,param,n_jobs=-1)

# random seed
np.random.seed(123)


## START TIME
start = time()
# Initialize the design with some (i.e. 3) initial random evaluations
bo._firstRun(n_eval=initialPoints)

for iter in range(furtherEvaluations):
    bo._optimizeAcq(method='L-BFGS-B', n_start=1)
    bo.updateGP()
## END TIME
end = time() - start

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

In [18]:
print('Execution time: {}\nBayesan Optimization Results: {}'.format(end, bo.getResult()))

Execution time: 5.923130512237549
Bayesan Optimization Results: (OrderedDict([('C', 0.10058849729597702), ('gamma', 8.235954788764426), ('coef0', 3.626774143406337)]), 0.8700012899896802)


### Surrogate model: Random Forest
### Acquisition function: UCB

In [19]:
# creating a GP surrogate model with a Squared Exponantial covariance function,
# aka kernel
sur_model = RandomForest()

# setting the acquisition function
acq = Acquisition(mode="UCB")

# creating an object Bayesian Optimization
bo = GPGO(sur_model,acq,compute_accuracy_SVC,param,n_jobs=-1)

# random seed
np.random.seed(123)


## START TIME
start = time()
# Initialize the design with some (i.e. 3) initial random evaluations
bo._firstRun(n_eval=initialPoints)

for iter in range(furtherEvaluations):
    bo._optimizeAcq(method='L-BFGS-B', n_start=1)
    bo.updateGP()
## END TIME
end = time() - start

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\done-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

In [20]:
print('Execution time: {}\nBayesan Optimization Results: {}'.format(end, bo.getResult()))

Execution time: 5.832818031311035
Bayesan Optimization Results: (OrderedDict([('C', 0.10058849729597702), ('gamma', 8.235954788764426), ('coef0', 3.626774143406337)]), 0.8700012899896802)


## Grid search

In [22]:
c_v = np.linspace(param['C'][1][0], param['C'][1][1], 5)              #si crea un array con valore min 0.1 e 
                                                                      #massimo 5 composto da 5 elementi per C
g_v = np.linspace(param['gamma'][1][0], param['gamma'][1][1], 6)      #si crea un array con valore min 0.1 e 
                                                                      #massimo 10 composto da 5 elementi per gamma
c0_v = np.linspace(param['coef0'][1][0], param['coef0'][1][0], 5)     #si crea un array con valore min 0.1 e 
                                                                      #massimo 10 composto da 5 elementi per coef0

results = []
for c in c_v:
    for g in g_v:
        for c0 in c0_v:
            results.append(compute_accuracy_SVC(c, g, c0))           #Vengono iterati i diversi parametri e si calcola la 
                                                                     #performance per ogni combinazione di parametri

In [23]:
results

[0.97218782249742,
 0.97218782249742,
 0.97218782249742,
 0.97218782249742,
 0.97218782249742,
 0.8811468008255933,
 0.8811468008255933,
 0.8811468008255933,
 0.8811468008255933,
 0.8811468008255933,
 0.8546826625386996,
 0.8546826625386996,
 0.8546826625386996,
 0.8546826625386996,
 0.8546826625386996,
 0.8491271069831441,
 0.8491271069831441,
 0.8491271069831441,
 0.8491271069831441,
 0.8491271069831441,
 0.8488003095975231,
 0.8488003095975231,
 0.8488003095975231,
 0.8488003095975231,
 0.8488003095975231,
 0.8602382180942552,
 0.8602382180942552,
 0.8602382180942552,
 0.8602382180942552,
 0.8602382180942552,
 0.9557383040935672,
 0.9557383040935672,
 0.9557383040935672,
 0.9557383040935672,
 0.9557383040935672,
 0.8271628826969384,
 0.8271628826969384,
 0.8271628826969384,
 0.8271628826969384,
 0.8271628826969384,
 0.7945239938080495,
 0.7945239938080495,
 0.7945239938080495,
 0.7945239938080495,
 0.7945239938080495,
 0.7780744754041967,
 0.7780744754041967,
 0.7780744754041967,
 0

In [24]:
len(results)

150